## Selección de hiper-parámetros

La mayoria de estimadores tienen hiper-parámetros que hay que ajustar para que el rendimiento sea bueno. Por ejemplo ya hemos visto el parámetro $k$ de vecinos próximos:

<img src=https://amueller.github.io/ml-workshop-2-of-4/slides/images/knn_boundary_varying_k.png width=500>

En la mayoría de estimadores estos hiper-parámetros representan la "flexibilidad" del modelo:

   * Modelos muy flexibles son capaces de memorizar el conjunto de entrenamiento, pero tendrán un mal rendimiento en el conjunto de test (sobreajuste)
   * Modelos poco flexibles no serán capaces de aprender el patrón de los datos, y tendrán un mal rendimiento en general (infraajuste)
   
Este equilibrio entre sobreajuste e infraajuste se suele representar de manera teórica con gráficos como este:

<img src=https://amueller.github.io/ml-workshop-2-of-4/slides/images/overfitting_underfitting_cartoon_full.png width=500>

Es importante destacar que no es un gráfico realizado con datos reales, sino que es un esquema de lo que se suele observar en la práctica. Hasta el momento dividíamos nuestros datos en dos conjuntos:

   * entrenamiento
   * test
   
<img src=https://amueller.github.io/ml-workshop-2-of-4/slides/images/train_test_split_new.png width=500>

Pero el conjunto de test **no** se puede usar para comparar el mismo modelo con distintos hiper-parámetros y elegir el que mejor resultado tenga. El conjunto de test **solo** se usa para dar una estimación del rendimiento final. 

### Conjunto de validación

Por tanto, para ajustar hiper-parámetros vamos a dividir los datos en tres conjuntos:

  * entramiento
  * test
  * validación
  
<img src=https://amueller.github.io/ml-workshop-2-of-4/slides/images/train_test_validation_split.png width=500>

In [1]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

ds = load_breast_cancer()

In [2]:
X_train_val, X_test, y_train_val, y_test = train_test_split(ds.data, ds.target, stratify=ds.target, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, stratify=y_train_val, random_state=0)

In [3]:
print(X_train.shape)
print(X_test.shape)
print(X_val.shape)

(319, 30)
(143, 30)
(107, 30)


In [4]:
from sklearn.neighbors import KNeighborsClassifier

val_scores = {}
for k in (1, 3, 5, 10, 15):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    val_scores[k] = knn.score(X_val, y_val)

In [5]:
import pandas as pd
val_series = pd.Series(val_scores)
val_series

1     0.925234
3     0.915888
5     0.915888
10    0.943925
15    0.925234
dtype: float64

In [6]:
k_best = val_series.idxmax()

knn = KNeighborsClassifier(n_neighbors=k_best)
knn.fit(X_train_val, y_train_val)

print(f'k óptimo: {k_best}')
print(f'acierto entrenamiento: {knn.score(X_train_val, y_train_val):.3f}')
print(f'acierto test: {knn.score(X_test, y_test):.3f}')

k óptimo: 10
acierto entrenamiento: 0.946
acierto test: 0.916


### Validación cruzada

Consiste en partir los datos de entrenamiento en varios subconjuntos e ir rotando el conjunto de validación:

<img src=../../img/cross_validation_new.png width=500>

Ventajas con respecto a tener un único conjunto de validación:

   * Más estable
   * Se usan todos los datos de entrenamiento para buscar los parámetros óptimos
   
Desventajas:

   * Más lento, hay que ajustar tantos modelos como subconjuntos

In [7]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

cv_scores = {}
for k in (1, 3, 5, 10, 15):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train_val, y_train_val)
    cv_scores[k] = np.mean(cross_val_score(knn, X_train_val, y_train_val, cv=10))

In [8]:
import pandas as pd
cv = pd.Series(cv_scores)
cv

1     0.924806
3     0.938926
5     0.943632
10    0.941307
15    0.934275
dtype: float64

#### Búsqueda en rejilla + validación cruzada

La estrategia de buscar los hiper-parámetros óptimos calculando el error de validación cruzada para cada valor de los parámetros en una rejilla es bastante habitual.

scikit-learn implementa esta estrategia en la clase `GridSearchCV`, que simplifica el proceso anterior:

In [9]:
from sklearn.model_selection import GridSearchCV

param_grid = {'n_neighbors': np.arange(1, 20, 2)}
cv = GridSearchCV(KNeighborsClassifier(), param_grid=param_grid, cv=10)
cv.fit(X_train_val, y_train_val)

GridSearchCV(cv=10, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': array([ 1,  3,  5,  7,  9, 11, 13, 15, 17, 19])})

In [10]:
import pandas as pd

pd.DataFrame(cv.cv_results_).sort_values(by='mean_test_score', ascending=False)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
2,0.000897,0.000139,0.002183,0.000179,5,{'n_neighbors': 5},0.930233,0.953488,0.953488,0.953488,0.953488,0.930233,0.928571,0.976190,0.928571,0.928571,0.943632,0.015774,1
5,0.000796,0.000032,0.002055,0.000091,11,{'n_neighbors': 11},0.930233,0.953488,0.930233,0.953488,0.953488,0.906977,0.928571,1.000000,0.928571,0.928571,0.941362,0.024084,2
3,0.000812,0.000038,0.002089,0.000239,7,{'n_neighbors': 7},0.930233,0.953488,0.953488,0.953488,0.953488,0.906977,0.928571,0.976190,0.928571,0.928571,0.941307,0.018969,3
4,0.000832,0.000070,0.002156,0.000136,9,{'n_neighbors': 9},0.930233,0.953488,0.930233,0.953488,0.953488,0.930233,0.928571,0.976190,0.928571,0.928571,0.941307,0.015864,3
8,0.000818,0.000107,0.002181,0.000172,17,{'n_neighbors': 17},0.930233,0.953488,0.930233,0.953488,0.930233,0.906977,0.952381,0.976190,0.928571,0.928571,0.939037,0.018589,5
1,0.000827,0.000044,0.002264,0.000376,3,{'n_neighbors': 3},0.930233,0.930233,0.953488,0.953488,0.953488,0.930233,0.928571,0.976190,0.904762,0.928571,0.938926,0.019061,6
9,0.000786,0.000013,0.002094,0.000071,19,{'n_neighbors': 19},0.930233,0.953488,0.930233,0.953488,0.930233,0.883721,0.952381,0.952381,0.928571,0.928571,0.934330,0.020159,7
6,0.000784,0.000011,0.002056,0.000115,13,{'n_neighbors': 13},0.930233,0.953488,0.930233,0.953488,0.930233,0.906977,0.928571,0.952381,0.928571,0.928571,0.934275,0.013993,8
7,0.000808,0.000063,0.002034,0.000096,15,{'n_neighbors': 15},0.930233,0.953488,0.930233,0.953488,0.930233,0.906977,0.928571,0.952381,0.928571,0.928571,0.934275,0.013993,8
0,0.001045,0.000243,0.002775,0.000996,1,{'n_neighbors': 1},0.930233,0.953488,0.906977,0.953488,0.883721,0.953488,0.880952,1.000000,0.880952,0.904762,0.924806,0.037955,10


In [11]:
print(f'k óptimo: {cv.best_params_}')
print(f'mejor cv score: {cv.best_score_:.3f}')
print(f'acierto test: {cv.score(X_test, y_test):.3f}')

k óptimo: {'n_neighbors': 5}
mejor cv score: 0.944
acierto test: 0.916


### Otras estrategias de validación cruzada

La estrategia anterior se conoce con el nombre de validación cruzada de $k$ hojas o $k$-fold cross-validation. Como hemos visto, consiste en crear $k$ subconjuntos aleatorios de forma aleatoria con probabilidad uniforme.

También existen otras estrategias:

   1. `StratifiedKFold`, generar los subconjuntos de forma que se mantengan la proporción de las clases (estratificados). Esto es especialmente importante si las clases **no están balanceadas**. En `GridSearchCV` y `cross_val_score` la validación cruzada está **estratificada** por defecto si el estimador es de clasificación
   
   2. `LeaveOneOut`: validación cruzada de $k$ hojas con $k=1$
   
   3. `(Stratified)ShuffleSplit`: muestrea el conjunto de test con reemplazamiento
   
   4. `Repeated(Stratified)KFold`: repite la validación cruzada múltiples veces

#### Validación cruzada en series temporales

Para las series temporales no tiene sentido escoger el conjunto de test de manera aleatoria, ya que estamos "prediciendo" usando datos del futuro
 
Conjunto de test aleatorio | Conjunto de test estructurado
---|---
<img src=https://amueller.github.io/ml-workshop-2-of-4/slides/images/time_series2.png style="width:100%"> | <img src=https://amueller.github.io/ml-workshop-2-of-4/slides/images/time_series3.png style="width:100%">

La clase `TimeSeriesSplit` realiza una validación cruzada pero manteniendo la estructura temporal:

<img src=https://amueller.github.io/ml-workshop-2-of-4/slides/images/time_series_cv.png width=500>

De esta forma, si por ejemplo tenemos datos de 1 año:

   * *Primera partición*: Enero-Octubre entrenamiento, Noviembre-Diciembre test
   * *Segunda partición*: Enero-Agosto entrenamiento, Septiembre-Octubre test
   * *Tercera partición*: Enero-Junio entrenamiento, Julio-Agosto test
   * etc
   
Es importante destacar que el conjunto de test siempre tiene el mismo tamaño (2 meses en este caso), pero el conjunto de entrenamiento tiene tamaño variable.

In [12]:
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold

cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=10)

param_grid = {'n_neighbors': np.arange(1, 20, 2)}
grid = GridSearchCV(KNeighborsClassifier(), param_grid=param_grid, cv=cv)
grid.fit(X_train_val, y_train_val)

print(f'k óptimo: {grid.best_params_}')
print(f'mejor cv score: {grid.best_score_:.3f}')
print(f'acierto test: {grid.score(X_test, y_test):.3f}')

k óptimo: {'n_neighbors': 7}
mejor cv score: 0.936
acierto test: 0.923


### Ejercicios

Con el conjunto de datos de titanic:

   1. Preparar los datos de la misma forma que en el notebook `03-missing.ipynb`, imputando los valores que faltan de `age` con la media
   
   2. Ajustar un modelo `LogisticRegression` y buscar el valor óptimo del parámetro `C`